In [2]:
# rSLDS actual data 

In [ ]:
# RUN IN COLAB ONLY: install required modules: ssm, openpyxl
!pip -q install git+https://github.com/lindermanlab/ssm.git #egg=ssm
!pip -q install openpyxl

In [ ]:
# RUN IN COLAB ONLYL: force-fetch latest files from GitHub (overwrite local copies)
!curl -L -H "Cache-Control: no-cache" -o gridsearch.py https://raw.githubusercontent.com/chrismader1/SLDS/main/gridsearch.py
!curl -L -H "Cache-Control: no-cache" -o rSLDS.py https://raw.githubusercontent.com/chrismader1/SLDS/main/rSLDS.py

In [ ]:
# RUN IN COLAB ONLYL: create a subfolder /Data for manual transfer 
import os
os.makedirs("Data", exist_ok=True)
# Copy-paste the following files into the Data subfolder:
# > bbg_data.xlsx
# > F-F_Momentum_Factor_daily.csv
# > F-F_Research_Data_5_Factors_2x3_daily.csv
# > F-F_Research_Data_Factors_daily.csv

In [4]:
import numpy as np
import pandas as pd
# sys.path.insert(0, "/Users/chrismader/Python/SLDS")
from gridsearch import pipeline_actual

In [8]:
# --------------------------------------------------------------------------------------
# CONFIG
# --------------------------------------------------------------------------------------

CONFIG = {
    
    # Core defaults
    "n_jobs": -1,  # multi-threading
    "dt": 1.0 / 252.0,
    "n_iters": 10,
    "h_z": 0.0,  # no CUSUM overlay for actual data
    
    # Batch windows
    "batch_grid": [
        {"train_window": 756, "overlap_window": 10},
        # {"train_window": 256, "overlap_window": 63},
        # {"train_window": 504, "overlap_window": 63},
        # {"train_window": 756, "overlap_window": 63},
        # {"train_window": 1260, "overlap_window": 63},
    ],

    # Number of regimes
    "K_grid": [3, 4],
    # "K_grid": [2],
    
    # Unrestricted models: 
    "unrestricted_models": [
       {"label": "[y]",         "channels": ["y"],                "dim_latent": [1]},
        {"label": "[y,h]",       "channels": ["y","h"],            "dim_latent": [2]},
        {"label": "[g,v]",       "channels": ["g","v"],            "dim_latent": [2]},
        {"label": "[g,v,h]",     "channels": ["g","v","h"],        "dim_latent": [2,3]},
        {"label": "[y,g,v,h]",   "channels": ["y","g","v","h"],    "dim_latent": [3,4]},
    ],

    # Restricted models: 
    "restricted_models": [
        {"label": "fund1",        "channels": ["y"],                 "dim_latent": [2],    "C_type": "fund1"},
        {"label": "fund1_vix",    "channels": ["y","h"],             "dim_latent": [3],    "C_type": "fund1_vix"},
        {"label": "fund2",        "channels": ["y","g"],             "dim_latent": [2],    "C_type": "fund2"},
        {"label": "fund2_vix",    "channels": ["y","g","h"],         "dim_latent": [3],    "C_type": "fund2_vix"},
        {"label": "fund3",        "channels": ["y","v","g"],         "dim_latent": [2],    "C_type": "fund3"},
        {"label": "fund3_vix",    "channels": ["y","v","g","h"],     "dim_latent": [3],    "C_type": "fund3_vix"},   

        {"label": "factor1",      "channels": ["y"],                 "dim_latent": [2],    "C_type": "factor1"},
        {"label": "factor1_vix",  "channels": ["y","h"],             "dim_latent": [3],    "C_type": "factor1_vix"},

        {"label": "factor2_ff3",   "channels": ["y","mkt","smb","hml"],                   "dim_latent": [3], "C_type": "factor2"},
        {"label": "factor2_ff3mom","channels": ["y","mkt","smb","hml","mom"],             "dim_latent": [4], "C_type": "factor2"},
        {"label": "factor2_ff5",   "channels": ["y","mkt","smb","hml","rmw","cma"],       "dim_latent": [5], "C_type": "factor2"},
        {"label": "factor2_ff5mom","channels": ["y","mkt","smb","hml","rmw","cma","mom"], "dim_latent": [6], "C_type": "factor2"},
    ],

    # Model selection
    "run_unrestricted": True,
    "run_restricted": True,

    # Output
    "display": True,
    "verbose": True,    
}


In [10]:
# --------------------------------------------------------------------------------------
# Execute
# --------------------------------------------------------------------------------------

def main():

    # filename = "/Users/chrismader/Python/SLDS/Data/bbg_data.xlsx"
    filename = "Data/bbg_data.xlsx"
    
    # df = pd.read_excel(filename, sheet_name="SPX_PX", skiprows=4, index_col=0)
    # df = df.iloc[3:, :]
    # securities = df.columns.to_list()
    
    # securities = ['MSFT', 'NVDA', 'AAPL', 'AMZN', 'META', 'AVGO', 'GOOGL', 'TSLA']

    # securities = ['AMD']
    
    securities = [
        "MSFT","NVDA","AAPL","AVGO","GOOGL","ORCL","IBM","CSCO","CRM","AMD","INTU","NOW","TXN","QCOM",
        "ADBE","AMAT","PLTR","ACN","META","NFLX","GOOGL","GOOG","DIS","VZ","T","UBER","AMZN","TSLA","HD",
        "MCD","BKNG","COST","WMT","PG","KO","PM","PEP","BRK/B","JPM","V","MA","BAC","WFC","GS","MS","SPGI",
        "SCHW","AXP","BLK","C","PGR","LLY","JNJ","ABBV","UNH","ABT","MRK","ISRG","BSX","TMO","AMGN","RTX",
        "XOM","CVX","GE","CAT","BA","HON","LIN","NEE","DUK","SO","AEP","D","AMT","PLD","EQIX","O","CBRE"]
    
    _ = pipeline_actual(securities=securities, CONFIG=CONFIG, filename=filename)

if __name__ == "__main__":
    main()


KeyboardInterrupt: 